# Loading a saved model and merging the saved adapter weights
* this requires more memory than what the training takes up
* instance ml.g4dn.2xlarge
* make sure your libraries that are getting installed here match the ones you used at training stage

In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops py7zr tensorboardX
!pip install -q -U torch==2.0.1 bitsandbytes==0.40.2


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


# GPU Check

In [7]:
!nvidia-smi

Mon Sep 25 03:28:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P0    26W /  70W |  14886MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Make sure the path is pointing to the correct checkpoint
* do not make any changes to the names of the files in the chek point direcotry

In [5]:
peft_model_id = "./results/checkpoint-20"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    return_dict=True, 
    load_in_8bit=True, 
    device_map={"":0},
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards: 100%|██████████| 8/8 [00:17<00:00,  2.14s/it]


In [6]:
model = model.merge_and_unload()
torch.save(model.state_dict(), './merged_model_and_adapter_weights')

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:68: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 316.00 MiB (GPU 0; 14.76 GiB total capacity; 12.80 GiB already allocated; 223.75 MiB free; 13.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF